# Part 3: Ranking

## Imports

In [1]:
import pandas as pd
from collections import defaultdict
import numpy as np
import re
import nltk
import contractions
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.manifold import TSNE
import matplotlib
import matplotlib.pyplot 

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Campero/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Campero/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/Campero/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
df = pd.read_csv('processed_tweets.csv').drop(columns=['Unnamed: 0'])

### From these results of part 2 we had concluded that some queries could be:

In [3]:
Q1="florida hurrican"

Q2="help people in florida"

Q3="hurrican ian major damages"

Q4="storm impact in Florida"

Q5="floodings in the south"

# a) Scoring using TF-IDF + Cosine Similarity 
Classical scoring, we have also seen during the practical labs

In [4]:
def extract_tweet_vocabulary(tweet, docId):
    return {term: docId for term in tweet.split(' ')}

In [5]:
def merge_dicts(dicts):
    vocab = defaultdict(list)
    for dic in dicts:
        for term in dic:
            vocab[term].append(dic[term])
    return dict(vocab)

In [6]:
tweets_dicts = map(extract_tweet_vocabulary, df['Tweet'], df['DocID'])
vocabulary = merge_dicts(tweets_dicts)
print(f"Vocabulary has {len(vocabulary)} words")

Vocabulary has 10671 words


In [7]:
def build_TF_IDF(df, vocabulary):
    terms = vocabulary.keys()
    docs = df.DocID
    N = len(docs)
    tf_idf = dict()
    
    for doc in docs:
        tf_idf[doc] = {}
    
    for term in terms:
        for doc in vocabulary[term]:
            tf = df[df.DocID == doc].Tweet.iloc[0].split().count(term)
            if tf>0:
                df_i = len(vocabulary[term])
                tf_idf[doc][term] = (1+np.log(tf))*np.log(N/df_i)
            else:
                tf_idf[doc][term] = 0
    return tf_idf

In [8]:
def find_doc2norm(tf_idf):
    docs = tf_idf.keys()
    doc2norm = {}
    for doc in docs:
        doc2norm[doc] = np.linalg.norm(np.array(list(tf_idf[doc].values())))
    return doc2norm

In [9]:
def find_term2sum(tf_idf):
    terms = vocabulary.keys()
    docs = tf_idf.keys()
    term2sum = {term:0 for term in terms}
    for doc in docs:
        for term, value in tf_idf[doc].items():
            term2sum[term] += value
    term2sum = {t: v for t, v in sorted(term2sum.items(), key=lambda item: item[1], reverse=True)}
    return term2sum

In [ ]:
tf_idf = build_TF_IDF(df, vocabulary)

In [ ]:
doc2norm = find_doc2norm(tf_idf)
term2sum = find_term2sum(tf_idf)

In [ ]:
def build_terms(line):
    """
    Preprocess the tweet content removing stop words, contractionas and urls
    lemmatizing and stemming words to keep a single word for each family of words
    transforming in lowercase, removing special characters [#, @, .] 
    (since it is included in another column on the dataframe)
    
    return tokenized tweet (list of words after applying the previous steps).
    
    Argument:
    line -- string (tweet) to be preprocessed
    
    Returns:
    line - a list of tokens corresponding to the input text after the preprocessing
    """
    ## START CODE
    line = line.lower() ##Transform in lowercase
    line = re.sub(r"[^A-Za-z 0-9 ']+", '', line) # remove emojis and any other special character
    stop_words = set(stopwords.words("english")) # removing stopwords
    line = ' '.join([contractions.fix(x) for x in line.split(' ')]) # expaning verb abreviations: i'll -> i will 
    line = re.sub("'", '', line) 
    line = line.split(' ')
    line = [x for x in line if x and x not in stop_words]
    line = filter(lambda x:x[0:5]!='https', line) # removing links
    line = [x for x in line]
    ps = PorterStemmer() 
    lemmatizer = WordNetLemmatizer() 
    line = [lemmatizer.lemmatize(x) for x in line] # keeping the singular form of each noun: feet --> foot
    line = [ps.stem(x) for x in line] # keeping the root of each family of words: dancer --> danc
    
    ## END CODE
    return ' '.join(line)

In [ ]:
def rank(query, tf_idf, vocabulary):
    N = len(tf_idf.keys())
    terms_query = build_terms(query).split()
    terms_q = list(set(terms_query))
    tf_idf_q = dict()
    
    for term_q in terms_q:
        f_iq = terms_query.count(term_q)
        if term_q not in vocabulary:
            continue
        df_i = len([doc for doc in vocabulary[term_q] if doc in tf_idf.keys()])
        if df_i>0:
            tf_idf_q[term_q] = (1+np.log(f_iq))*np.log(N/df_i)
        else:
            tf_idf_q[term_q]=0
    q_norm = np.linalg.norm(np.array(list(tf_idf_q.values())))
    doc2score = dict()
    
    doc2norm = find_doc2norm(tf_idf)
    
    for doc, dix in tf_idf.items():
        dot_product = 0
        for term, value in dix.items():
            if term in tf_idf_q.keys():
                dot_product += value * tf_idf_q[term]
        doc2score[doc] = dot_product / (doc2norm[doc] * q_norm)
    doc2score = {t: v for t, v in sorted(doc2score.items(), key=lambda item: item[1], reverse=True)}
    return doc2score

In [ ]:
doc2score_Q1 = rank(Q1, tf_idf, vocabulary)
top = 10
print(f"Top {top} docs for query 1 ('{Q1}'):")
for i in range(top):
    doc, score = list(doc2score_Q1.items())[i]
    print(f"\t {i+1}) {doc} with a score of {score} and with the following information:\n{df[df.DocID == doc].iloc[0]}")

In [ ]:
doc2score_Q2 = rank(Q2, tf_idf, vocabulary)
top = 10
print(f"Top {top} docs for query 1 ('{Q2}'):")
for i in range(top):
    doc, score = list(doc2score_Q2.items())[i]
    print(f"\t {i+1}) {doc} with a score of {score} and with the following information:\n{df[df.DocID == doc].iloc[0]}")

In [ ]:
doc2score_Q3 = rank(Q3, tf_idf, vocabulary)
top = 10
print(f"Top {top} docs for query 1 ('{Q3}'):")
for i in range(top):
    doc, score = list(doc2score_Q3.items())[i]
    print(f"\t {i+1}) {doc} with a score of {score} and with the following information:\n{df[df.DocID == doc].iloc[0]}")

In [ ]:
doc2score_Q4 = rank(Q4, tf_idf, vocabulary)
top = 10
print(f"Top {top} docs for query 1 ('{Q4}'):")
for i in range(top):
    doc, score = list(doc2score_Q4.items())[i]
    print(f"\t {i+1}) {doc} with a score of {score} and with the following information:\n{df[df.DocID == doc].iloc[0]}")

In [ ]:
doc2score_Q5 = rank(Q5, tf_idf, vocabulary)
top = 10
print(f"Top {top} docs for query 1 ('{Q5}'):")
for i in range(top):
    doc, score = list(doc2score_Q5.items())[i]
    print(f"\t {i+1}) {doc} with a score of {score} and with the following information:\n{df[df.DocID == doc].iloc[0]}")

### Take into account that for future queries, the final output must return (when
present) the following information for each of the selected documents: Tweet |
Username | Date | Hashtags | Likes | Retweets | Url (here the “Url” means the
tweet link).

# b) Own score + cosine similarity

Here the task is to create a new score, and it’s up to you to create a new one.


In [ ]:
doc2extra_score = dict()

for index, row in df.iterrows():
    extra_score = (row["Likes"] + row["Retweets"]) / len(row["Hashtags"].split(","))
    doc2extra_score[row["DocID"]] = extra_score

mu = np.mean(list(doc2extra_score.values()))
doc2extra_score = {k: v / mu for k, v in doc2extra_score.items()}

In [ ]:
def new_rank(query, tf_idf, vocabulary, doc2extra_score):
    N = len(tf_idf.keys())
    terms_query = build_terms(query).split()
    terms_q = list(set(terms_query))
    tf_idf_q = dict()
    
    for term_q in terms_q:
        f_iq = terms_query.count(term_q)
        if term_q not in vocabulary:
            continue
        df_i = len([doc for doc in vocabulary[term_q] if doc in tf_idf.keys()])
        if df_i>0:
            tf_idf_q[term_q] = (1+np.log(f_iq))*np.log(N/df_i)
        else:
            tf_idf_q[term_q]=0
    q_norm = np.linalg.norm(np.array(list(tf_idf_q.values())))
    doc2score = dict()
    
    doc2norm = find_doc2norm(tf_idf)
    
    for doc, dix in tf_idf.items():
        dot_product = 0
        for term, value in dix.items():
            if term in tf_idf_q.keys():
                dot_product += value * tf_idf_q[term]
        doc2score[doc] = dot_product / (doc2norm[doc] * q_norm) + doc2extra_score[doc]
    doc2score = {t: v for t, v in sorted(doc2score.items(), key=lambda item: item[1], reverse=True)}
    return doc2score

In [ ]:
doc2score_Q1 = new_rank(Q1, tf_idf, vocabulary, doc2extra_score)
top = 10
print(f"Top {top} docs for query 1 ('{Q1}'):")
for i in range(top):
    doc, score = list(doc2score_Q1.items())[i]
    print(f"\t {i+1}) {doc} with a score of {score} and with the following information:\n{df[df.DocID == doc].iloc[0]}")

In [ ]:
doc2score_Q2 = new_rank(Q2, tf_idf, vocabulary, doc2extra_score)
top = 10
print(f"Top {top} docs for query 2 ('{Q2}'):")
for i in range(top):
    doc, score = list(doc2score_Q2.items())[i]
    print(f"\t {i+1}) {doc} with a score of {score} and with the following information:\n{df[df.DocID == doc].iloc[0]}")

In [ ]:
doc2score_Q3 = new_rank(Q3, tf_idf, vocabulary, doc2extra_score)
top = 10
print(f"Top {top} docs for query 3 ('{Q3}'):")
for i in range(top):
    doc, score = list(doc2score_Q3.items())[i]
    print(f"\t {i+1}) {doc} with a score of {score} and with the following information:\n{df[df.DocID == doc].iloc[0]}")

In [ ]:
doc2score_Q4 = new_rank(Q4, tf_idf, vocabulary, doc2extra_score)
top = 10
print(f"Top {top} docs for query 4 ('{Q4}'):")
for i in range(top):
    doc, score = list(doc2score_Q4.items())[i]
    print(f"\t {i+1}) {doc} with a score of {score} and with the following information:\n{df[df.DocID == doc].iloc[0]}")

In [ ]:
doc2score_Q5 = new_rank(Q5, tf_idf, vocabulary, doc2extra_score)
top = 10
print(f"Top {top} docs for query 5 ('{Q5}'):")
for i in range(top):
    doc, score = list(doc2score_Q5.items())[i]
    print(f"\t {i+1}) {doc} with a score of {score} and with the following information:\n{df[df.DocID == doc].iloc[0]}")

# c) BM25

In [ ]:
x = {1: 2, 3: 4, 4: 3, 2: 1, 0: 0}
a = {k: v for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)}
print(a)

In [ ]:
def BM25(query, vocabulary, k1, b, L_ave, doc_contents):
    '''
    vocabulary = inverted index, dictionary with keys = terms an values = list of doc_ids
    k1 = value to regulate xx
    b = value to regulate yy
    L_ave = average length of docs in words
    doc_contents = dictionary where keys = doc_ids and values = list of terms (after text processing)
    '''
    
    RSV = dict()
    N = len(doc_contents)
    terms_q = list(set(build_terms(query).split()))
    idf = dict()
    
    # calculate idf for each term in the query 
    for t in terms_q:
        f_tq = terms_q.count(t)
        if t not in vocabulary:
            continue
        df_t = len(vocabulary[t])
        idf[t] = np.log(N/df_t)
        
    # calculate RSVd for each document  
    for doc in doc_contents.keys():
        RSV[doc] = 0
        Ld = len(doc_contents[doc])
        for t in idf.keys():
            tf_t_d = doc_contents[doc].count(t)

            second_term = ((k1+1)*tf_t_d) / (k1*((1-b)+b*(Ld/L_ave))+tf_t_d)
            RSV[doc] += idf[t]*second_term
            
    return {k: v for k, v in sorted(RSV.items(), key=lambda item: item[1], reverse=True)}
            

In [ ]:
k1 = 1
b = 1
L_ave = np.mean([len(x.split()) for x in df.Tweet])
print(L_ave)
dictionary_doc = df.copy().drop(columns=['Username', 'Date', 'Hashtags', 'Likes', 'Retweets', 'Url'])
dictionary_doc = dictionary_doc.set_index('DocID').T.to_dict('list')
dictionary_doc = {k: x[0].split() for k, x in dictionary_doc.items()}
doc2score_Q5 = BM25(Q5, vocabulary, k1, b, L_ave, dictionary_doc)
top = 10
print(f"Top {top} docs for query ('{Q5}'):")
for i in range(top):
    doc, score = list(doc2score_Q5.items())[i]
    print(f"\t {i+1}) {doc} with a score of {score} and with the following information:\n{df[df.DocID == doc].iloc[0]}")

# 2. Word2vec + cosine similarity

In [ ]:
import gensim
from gensim.models import Word2Vec
from numpy import dot
from numpy.linalg import norm

In [ ]:
# storing doc words in a list of lists 
docs_as_words = []
for k in dictionary_doc.keys():
    docs_as_words.append(dictionary_doc[k])

In [ ]:
# training model for word2vec
model1 = gensim.models.Word2Vec(docs_as_words, min_count = 1,
                              vector_size = 100, window = 5)

In [ ]:
# function to convert any text to a vector
def text2vec(text):

    #compute a vector for each word in the text  
    vectors = []
    for w in text:
        vector = model1.wv[w]
        vectors.append(vector)

    # sum all vectors previously computed
    doc_to_vec = np.zeros(len(vectors[0]))
    for v in vectors: 
        doc_to_vec += v

    # take average of previous sum
    text_2_vec = doc_to_vec/len(content) 
        
    return text_2_vec

In [ ]:
# converting all tweets to vectors
tweets_to_vecs = {}
for doc, content in dictionary_doc.items():
    tweets_to_vecs[doc] = text2vec(content)

In [ ]:
# processing queries
queries = [Q1, Q2, Q3, Q4, Q5]
processed_queries = [build_terms(q) for q in queries]

# removing words that don't appear in our vocabulary 
clean_queries = [[w for w in q.split(' ') if w in vocabulary.keys()] for q in queries]

# converting all queries to vectors
queries_to_vecs = {}
i=0
for q in clean_queries: 
    queries_to_vecs[f'Q{i}'] = text2vec(q)
    i+=1

In [ ]:
def cosine_similarity(vec1, vec2):
    cos_sim = dot(vec1, vec2) / (norm(vec1) * norm(vec2))
    return cos_sim

In [ ]:
# compute cos similarities for each (query, doc) pair and sort them 
cos_similarities = {}
for query, query_vec in queries_to_vecs.items():
    tweet_queries_sim = {}
    for tweet, tweet_vec in tweets_to_vecs.items():
        tweet_queries_sim[tweet] = cosine_similarity(tweet_vec, query_vec)
        
    tweet_queries_sim = {k: v for k, v in sorted(tweet_queries_sim.items(), key=lambda item: item[1], reverse=True)}
    cos_similarities[query] = tweet_queries_sim

In [ ]:
top_20_docs = {}
for query, docs in cos_similarities.items(): 
    i=0
    top_20_docs[query] = []
    for doc in docs.keys():
        if i<20:
            top_20_docs[query].append(doc)
            i+=1
        else: 
            break 
    

In [ ]:
print(top_20_docs)

In [ ]:
top = 20

i=0
for q in queries:
    print(f"Top {top} docs for query ('{q}'):")
    j=0
    for doc in top_20_docs[f'Q{i}']:
        score = cos_similarities[f'Q{i}'][doc]
        print(f"\t {j+1}) {doc} with a score of {score} and with the following information:\n{df[df.DocID == doc].iloc[0]}")
        j+=1
    i+=1